In [ ]:
"""
To work with alpaca you will need to make an account and get your key and secret. I have mine in a separate file that is gitignored and not pushed. 
Eventually we should figure out how to automate this refresh process as as of now I am doing it manually.

API key and secret can be found in the "home" page of your alpaca account when you are logged in on the right side of the screen after scrolling a bit

Having wscat installed will also be very useful for testing straight from the command line

Additionally I have a virtual environment folder set up in this moby folder that has its own list of requirements. We can see if all packages are compatible 
and have one top level virtual environment later on but for now this was just done for the sake of simplicity
"""

In [1]:
"""
Websocket streaming doc: https://docs.alpaca.markets/docs/streaming-market-data

Basic auth command with wscat: {"action": "auth", "key": "{KEY_ID}", "secret": "{SECRET}"

If authentication is successful you will get the message: [{"T":"success","msg":"authenticated"}]

Once you are authenticated, you can subscribe to a give symbol (ticker)

A subscription is essentially what allows us to listen for data on a given symbol and constantly stream data for it. 
With the free version we can listen to a maximum of 30 symbols at once.

To subscribe to one or more symbols you can use the following command after connecting:

{
  "action": "subscribe",
  "<channel1>": ["<SYMBOL1>"],
  "<channel2>": ["<SYMBOL2>","<SYMBOL3>"],
  "<channel3>": ["*"]
}

Example:

"""


'\nWebsocket streaming doc: https://docs.alpaca.markets/docs/streaming-market-data\n\nBasic auth command with wscat: {"action": "auth", "key": "{KEY_ID}", "secret": "{SECRET}"\n\nIf authentication is successful you will get the message: [{"T":"success","msg":"authenticated"}]\n\nOnce you are authenticated, you can subscribe to a give symbol (ticker)\n\nA subscription is essentially what allows us to listen for data on a given symbol and constantly stream data for it. \nWith the free version we can listen to a maximum of 30 symbols at once.\n\nTo subscribe to one or more symbols you can use the following command after connecting:\n\n{\n  "action": "subscribe",\n  "<channel1>": ["<SYMBOL1>"],\n  "<channel2>": ["<SYMBOL2>","<SYMBOL3>"],\n  "<channel3>": ["*"]\n}\n\nExample:\n\n'

Schemas and Data formatting info:

Stocks: https://docs.alpaca.markets/docs/real-time-stock-pricing-data

Options: https://docs.alpaca.markets/docs/real-time-option-data

News: https://docs.alpaca.markets/docs/real-time-stock-pricing-data

Crypto: https://docs.alpaca.markets/docs/real-time-crypto-pricing-data





In [6]:
from api_keys import api_key, api_secret
import numpy as np
import alpaca
import json
import websockets
import asyncio

In [ ]:
#The version of the streaming function below does not work in a jupyter notebook as it already runs an event loop. the version in the next cell has been slightly modified to work around this
#however when developing the project it will correspond to the version in this cell instead

alpaca_stream_url = "wss://stream.data.alpaca.markets/v2/iex"

async def stream_market_data(symbols_trades=None, symbols_quotes=None, symbols_bars=None):
    """
    each of the three parameters define which symbols we are getting the specified data for

    trades are just that, specific trades that have been placed on that given security 
    in the time frame

    quotes are the real-time latest price at which a security can be bought or sold.
    a quote consists of two prices, the bid (highes price a buyer will pay) and the ask
    (lowest price a seller will accept).

    Bars represent the 


    
    """
    async with websockets.connect(alpaca_stream_url) as ws:
        print("Openend connection")
        print(api_key)
        print(api_secret)
        #authenticate with api key and secret
        auth_message  ={
            "action" : "auth", 
            "key_id" : api_key,
            "secret_key" : api_secret
        }


        await ws.send(json.dumps(auth_message))

        #build the subscription message
        subscribe_message = {"action" : "subscribe"}
        if symbols_trades:
            subscribe_message["trades"] = symbols_trades
        if symbols_quotes:
            subscribe_message["quotes"] = symbols_quotes
        if symbols_bars:
            subscribe_message["bars"] = symbols_bars
        

        await ws.send(json.dumps(subscribe_message))

        print(f"Subscribed: {subscribe_message}")
        

        async for message in ws:
            data = json.loads(message)
            print(data)

symbols_trades = ["AAPL", "MSFT"]
symbols_quotes = ["AAPL", "MSFT"]
symbols_bars = ["AAPL"] 

asyncio.run(stream_market_data(symbols_trades, symbols_quotes, symbols_bars))

In [ ]:

import json
import websockets

alpaca_stream_url = "wss://stream.data.alpaca.markets/v2/iex"

async def stream_market_data_notebook(symbols_trades=None, symbols_quotes=None, symbols_bars=None):
    print("Connecting to Alpaca market data stream...")
    async with websockets.connect(alpaca_stream_url) as ws:
        
        #authenticate
        auth_message = {
            "action": "auth",
            "key":api_key,
            "secret": api_secret
        }
        await ws.send(json.dumps(auth_message))
        print("Sent auth message.")

        #build subscription message
        subscribe_msg = {"action": "subscribe"}
        if symbols_trades:
            subscribe_msg["trades"] = symbols_trades
        if symbols_quotes:
            subscribe_msg["quotes"] = symbols_quotes
        if symbols_bars:
            subscribe_msg["bars"] = symbols_bars

        await ws.send(json.dumps(subscribe_msg))
        print("Subscribed with:", subscribe_msg)

        # receive messages
        async for message in ws:
            data = json.loads(message)
            print(data)

#example symbols to track
symbols_trades = ["AAPL", "MSFT"]
symbols_quotes = ["AAPL", "MSFT"]
symbols_bars = ["AAPL"]  # minute bars if needed

# jupyter-safe execution
await stream_market_data_notebook(symbols_trades, symbols_quotes, symbols_bars)


Connecting to Alpaca market data stream...
Sent auth message.
Subscribed with: {'action': 'subscribe', 'trades': ['AAPL', 'MSFT'], 'quotes': ['AAPL', 'MSFT'], 'bars': ['AAPL']}
[{'T': 'success', 'msg': 'connected'}]
[{'T': 'success', 'msg': 'authenticated'}]
[{'T': 'subscription', 'trades': ['AAPL', 'MSFT'], 'quotes': ['AAPL', 'MSFT'], 'bars': ['AAPL'], 'corrections': ['AAPL', 'MSFT'], 'cancelErrors': ['AAPL', 'MSFT']}]


CancelledError: 

Openend connection
Subscribed: {'action': 'subscribe', 'trades': ['AAPL', 'MSFT'], 'quotes': ['AAPL', 'MSFT'], 'bars': ['AAPL']}
[{'T': 'success', 'msg': 'connected'}]
[{'T': 'error', 'code': 402, 'msg': 'auth failed'}]
[{'T': 'error', 'code': 401, 'msg': 'not authenticated'}]


ConnectionClosedError: no close frame received or sent

In [ ]:
# wss://stream.data.alpaca.markets/{version}/{feed}
#Version refers to the actual version of the service 
#feed refers to the actual place from which the information will be sourced
#see the following link for more information https://docs.alpaca.markets/docs/historical-stock-data-1#feed-parameter
#for the purposed of this demo we will use v2/iex as it is ideal for initial app testing and situations where precise pricing may not be the primary focus.
# It's a single US exchange that accounts for approximately ~2.5% of the market volume.


